# Обучение Qwen модели на искусственных задачах с GRPO

Сделано на основе ноутбука от [unsloth](https://unsloth.ai/blog/r1-reasoning)

Этот notebook показывает как обучать LLM на сгенерированных задачах из re-rl.

## Установка зависимостей

In [ ]:
!pip install unsloth vllm tensorboard trl
!pip install --upgrade pillow

## Импорты

In [ ]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

# Генераторы задач из re-rl
from re_rl.tasks.generators import (
    generate_random_linear_task,
    generate_random_quadratic_task,
    generate_random_futoshiki_task,
    generate_random_knights_knaves_task,
    generate_random_contradiction_task,
    generate_random_urn_probability_task,
    generate_random_arithmetic_task,
    ALL_TASK_GENERATORS,
)

# Физические задачи
from re_rl.tasks.physics.generators import (
    generate_random_kinematics_task,
    generate_random_quantum_task,
    generate_random_physics_task,
    ALL_PHYSICS_TASK_GENERATORS,
)

# Система наград
from re_rl.rewards import (
    reward_format_check,
    reward_cot_quality,
    reward_correctness,
)

# Промпты
from re_rl.tasks.prompts import PROMPT_TEMPLATES

## Генерация датасета для обучения

In [ ]:
from re_rl.dataset_generator import DatasetGenerator

generator = DatasetGenerator()

# Генерация SFT датасета
train_data = generator.generate_sft_dataset(
    task_types=["quadratic", "linear", "kinematics", "quantum", "arithmetic"],
    num_samples=5000,
    language="ru",
    difficulties=[3, 5, 7],
)

print(f"Сгенерировано {len(train_data)} примеров для обучения")

## Настройка модели

Используем LoRA для эффективного обучения на GPU с 24ГБ памяти.

In [ ]:
from unsloth import is_bfloat16_supported

max_seq_length = 2048
lora_rank = 64

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen2.5-1.5B-Instruct",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    fast_inference=True,
    max_lora_rank=lora_rank,
    gpu_memory_utilization=0.6,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=lora_rank,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=lora_rank,
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

## Подготовка датасета

In [ ]:
from datasets import Dataset

# Форматирование в chat формат
def format_for_training(example):
    messages = [
        {"role": "user", "content": f"{example['instruction']}\n\n{example['input']}"},
        {"role": "assistant", "content": example['output']}
    ]
    return {"messages": messages}

# Создаём датасет
dataset = Dataset.from_list([format_for_training(ex) for ex in train_data])
print(f"Датасет готов: {len(dataset)} примеров")

## Функции наград для GRPO

In [ ]:
import re

def extract_answer(text):
    """Извлекает ответ из текста."""
    patterns = [
        r"Ответ:\s*(.+?)(?:\n|$)",
        r"Answer:\s*(.+?)(?:\n|$)",
        r"=\s*([\d\.\-]+)\s*$",
    ]
    for pattern in patterns:
        match = re.search(pattern, text, re.MULTILINE | re.IGNORECASE)
        if match:
            return match.group(1).strip()
    return None

def reward_has_steps(text):
    """Награда за наличие шагов решения."""
    step_patterns = ["Шаг", "Step", "1.", "2.", "3."]
    has_steps = any(p in text for p in step_patterns)
    return 0.5 if has_steps else 0.0

def reward_has_answer(text):
    """Награда за наличие ответа."""
    answer = extract_answer(text)
    return 0.5 if answer else 0.0

def compute_total_reward(generated_text, expected_answer=None):
    """Вычисляет общую награду."""
    reward = 0.0
    reward += reward_has_steps(generated_text)
    reward += reward_has_answer(generated_text)
    
    # Проверка правильности если есть эталонный ответ
    if expected_answer:
        extracted = extract_answer(generated_text)
        if extracted and str(expected_answer) in extracted:
            reward += 1.0
    
    return reward

## Обучение с GRPO

In [ ]:
from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    output_dir="./grpo_output",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=5e-6,
    logging_steps=10,
    save_steps=500,
    max_new_tokens=512,
    num_generations=4,
)

# Примечание: полная настройка GRPO требует дополнительной конфигурации
# функции reward_fn в соответствии с документацией trl
print("Конфигурация GRPO готова")

## Альтернатива: SFT обучение

Если GRPO слишком сложен, можно начать с простого SFT.

In [ ]:
from trl import SFTTrainer, SFTConfig

sft_config = SFTConfig(
    output_dir="./sft_output",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=10,
    save_steps=500,
    max_seq_length=max_seq_length,
)

# trainer = SFTTrainer(
#     model=model,
#     train_dataset=dataset,
#     args=sft_config,
#     tokenizer=tokenizer,
# )
# trainer.train()

print("SFT конфигурация готова")

## Оценка модели

In [ ]:
def evaluate_on_tasks(model, tokenizer, num_tasks=10):
    """Оценивает модель на случайных задачах."""
    correct = 0
    
    for _ in range(num_tasks):
        # Генерируем случайную задачу
        task = generate_random_quadratic_task(language="ru")
        result = task.get_result()
        
        prompt = f"Решите задачу пошагово.\n\n{result['problem']}"
        
        # Генерируем ответ модели
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        outputs = model.generate(**inputs, max_new_tokens=512)
        generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Проверяем
        expected = str(result['final_answer'])
        if expected in generated:
            correct += 1
    
    accuracy = correct / num_tasks
    print(f"Accuracy: {accuracy:.2%} ({correct}/{num_tasks})")
    return accuracy

# evaluate_on_tasks(model, tokenizer, num_tasks=20)

## Сохранение модели

In [ ]:
# model.save_pretrained("./trained_model")
# tokenizer.save_pretrained("./trained_model")
# print("Модель сохранена!")